# GMM with/without a Weighting Matrix

In [1]:
using Optim

include("jlFiles/NWFn.jl")
include("jlFiles/printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
xx = readdlm("Data/FFmFactorsPs.csv",',',header=true)
xx = xx[1]
ym = xx[:,1]              #[yearmonth]
x  = xx[:,2]              #excess market returns
xx = nothing

T  = size(x,1)

388

## The Estimation Problem and the Test


If $x_{t}$ is $N(\mu,\sigma^{2})$, then the following moment conditions should
all be zero (in expectation)

$$
g_{t}(\mu,\sigma^{2})=\left[
\begin{array}
[c]{l}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}\\
(x_{t}-\mu)^{3}\\
(x_{t}-\mu)^{4}-3\sigma^{4}
\end{array}
\right]  .
$$

The first moment condition defines the mean $\mu$, the second defines the
variance $\sigma^{2}$, while the third and forth are the skewness and excess
kurtosis respectively.

## Traditional Estimation of Mean and Variance

In [3]:
mu = mean(x)                      #same as setting A*gbar=0
s2 = var(x)*(T-1)/T               #var() uses 1/(T-1) formula

par_a = [mu;s2]

println("\nParameters (col 1) and traditional std(parameters) in col 2:")
printmat([par_a [sqrt((s2/T));sqrt(2*s2^2/T)]])


Parameters (col 1) and traditional std(parameters) in col 2:
     0.602     0.233
    21.142     1.518



## GMM: A*g = 0



The following code from estimates the parameters (mean and
variance) by combining the 4 original moment conditions in $\bar{g}$ into 2
effective moment conditions, $A\bar{g}$, where $A$ is a $2\times4$ matrix

$$
A=\left[
\begin{array}
[c]{cccc}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{array}
\right]  
$$ 

This particular $A$ matrix implies that we use the classical
estimators of the mean and variance.

In [4]:
function Gmm4MomFn(par,x)

  mu = par[1]
  s2 = par[2]

  g = [(x-mu) ((x-mu).^2-s2) ((x-mu).^3) ((x-mu).^4-3*s2^2)]    #Tx4

  gbar = mean(g,1)'                         #4x1

  return g,gbar

end

Gmm4MomFn (generic function with 1 method)

In [5]:
println("\nGMM, exactly identified")

(g,gbar) = Gmm4MomFn(par_a,x)        #Tx4, moment conditions
q = size(g,2)
A = [1 0 0 0;                       #A in A*gbar=0 (here: all weight on first two moments)
     0 1 0 0]
println("\nChecking if mean of A*g_t = 0")
printmat(A*gbar)

D  = [-1                  0;                #Jacobian
      -2*mean(x-mu)      -1;
      -3*mean((x-mu).^2)   0;
      -4*mean((x-mu).^3)  -6*s2]
S  = NWFn(g,1)
V3 = inv(A*D)*A*S*A'inv(A*D)'

println("\nparameter, std(parameters)")
printmat([par_a sqrt.(diag(V3/T))])


GMM, exactly identified

Checking if mean of A*g_t = 0
    -0.000
     0.000


parameter, std(parameters)
     0.602     0.244
    21.142     2.381



In [6]:
function Gmm4MomLossFn(par,x,W=1)

  (g,gbar) = Gmm4MomFn(par,x)

  Loss = 1.0 + gbar'W*gbar                #to be minimized
  Loss = Loss[1]

  return Loss

end

Gmm4MomLossFn (generic function with 2 methods)

## GMM: g'Wg



Instead, the following code solves a minimization problem with the weighting matrix 

$$
W=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
$$ 

As a practical matter, it is often the case that a derivative-free method works better than other optimization routines.

In [7]:
println("\nGMM with weighting matrix")
                                          #gbar'W*gbar
W     = diagm([1;1;0;0])                  #weighting matrix
Sol   = optimize(par->Gmm4MomLossFn(par,x,W),par_a)

par_b = Optim.minimizer(Sol)

g,    = Gmm4MomFn(par_b,x)              #Tx4, moment conditions, evaluated at point estimates
S     = NWFn(g,1)                         #variance of sqrt(T)"gbar, NW with 1 lag
V2    = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)

println("Weighting matrix")
printmat(W)
println("\nparameter, std(parameters)")
printmat([par_b sqrt.(diag(V2/T))])


GMM with weighting matrix
Weighting matrix
         1         0         0         0
         0         1         0         0
         0         0         0         0
         0         0         0         0


parameter, std(parameters)
     0.602     0.244
    21.142     2.381



## GMM: g'Wg, Iterating over W


Finally, the following code iterates over the weighting
matrix by using $W=S^{-1}$, where the $S=\operatorname*{Cov}(\sqrt{T}\bar{g})$
is from the previous iteration.

In [8]:
println("\niterated GMM, using optimal weighting matrix, starting with S from previous estimation")

#W,S, par_c should be initialized outside loop to make them visible after it
par_c = par_b + 0.0
Dpar  = 1.0
i     = 1
println("\n\niterating over W starting with the W choice above")
while (Dpar > 1e-3) || (i < 2)    #require at least one iteration
  println("-------iteration  $i, old and new parameters--------")
  par_b           = par_c + 0.0     #important, par_b=par_c would make them always identical
  W               = inv(S)
  Sol             = optimize(par->Gmm4MomLossFn(par,x,W),par_b)   #use last estimates as starting point
  par_c           = Optim.minimizer(Sol)
  g,              = Gmm4MomFn(par_c,x)
  S               = NWFn(g,1)
  Dpar            = maximum(abs.(par_c-par_b))
  i               = i + 1
    printlnPs(par_b')
    printlnPs(par_c')
end

V2 = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)      #if non-optimal weighting matrix
V1 = inv(D'inv(S)*D)                      #with optimal weighting matrix

println("\nparameter, std_version2(parameters), std_version1(parameters)")
printmat([par_c sqrt.(diag(V2/T)) sqrt.(diag(V1/T))])


iterated GMM, using optimal weighting matrix, starting with S from previous estimation


iterating over W starting with the W choice above
-------iteration  1, old and new parameters--------
     0.602    21.142
     0.877    16.917
-------iteration  2, old and new parameters--------
     0.877    16.917
     0.879    16.647
-------iteration  3, old and new parameters--------
     0.879    16.647
     0.879    16.647

parameter, std_version2(parameters), std_version1(parameters)
     0.879     0.217     0.217
    16.647     1.311     1.311

